<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/gitlab_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gitlab
import yaml
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# GitLab configuration
GITLAB_URL = 'https://gitlab.com'  # GitLab instance URL
GITLAB_TOKEN = 'your_gitlab_token'  # GitLab personal access token
PROJECT_ID = 'your_project_id'  # The ID of the project to monitor

# Initialize GitLab client
gl = gitlab.Gitlab(GITLAB_URL, private_token=GITLAB_TOKEN)
project = gl.projects.get(PROJECT_ID)

# Function to get the latest pipeline status
def get_pipeline_status():
    pipelines = project.pipelines.list(order_by='id', sort='desc', per_page=1)
    if pipelines:
        pipeline = pipelines[0]
        return pipeline.id, pipeline.status, pipeline.detailed_status
    return None, None, None

# Function to fetch the failed job logs
def get_failed_job_logs(pipeline_id):
    jobs = project.pipelines.get(pipeline_id).jobs()
    failed_jobs = [job for job in jobs if job.status == 'failed']
    logs = {}
    for job in failed_jobs:
        logs[job.name] = job.trace()
    return logs

# Define the LangChain tool to interact with GitLab for analyzing errors
def analyze_pipeline_error(input_data):
    # Get pipeline status
    pipeline_id, status, detailed_status = get_pipeline_status()

    if status != 'failed':
        return f"Pipeline is successful! Status: {status}"

    # If the pipeline failed, get error logs
    if pipeline_id:
        logs = get_failed_job_logs(pipeline_id)
        return f"Pipeline failed! Detailed Status: {detailed_status}\nError Logs:\n{logs}"
    return "No failed pipeline found."

# Define the LangChain tool to fix Terraform code
def fix_terraform_code(input_data):
    # Get pipeline status
    pipeline_id, status, detailed_status = get_pipeline_status()

    if status != 'failed':
        return "Pipeline is successful, no errors to fix."

    if pipeline_id:
        logs = get_failed_job_logs(pipeline_id)
        # Try to fix the error by analyzing the Terraform error logs
        fixes = apply_terraform_fix(logs)
        return f"Pipeline failed! Attempting to fix errors. Applied fixes: {fixes}"

    return "No failed pipeline found."

# Function to apply fixes to Terraform code based on error logs
def apply_terraform_fix(error_logs):
    fixes = []

    # Example: fix for missing provider
    if "missing provider" in str(error_logs):
        fixes.append("Fixed missing provider issue in Terraform code.")
        update_terraform_with_provider()

    # Example: fix for incorrect syntax
    if "syntax error" in str(error_logs):
        fixes.append("Fixed syntax error in Terraform code.")
        update_terraform_syntax()

    # Return the applied fixes
    return fixes

# Function to update Terraform code with missing provider
def update_terraform_with_provider():
    # Fetch the Terraform file
    terraform_file = project.files.get(file_path='main.tf', ref='main')

    # Read the Terraform code
    terraform_code = terraform_file.decode()

    # Add the missing provider to the Terraform code (example fix)
    if 'provider "aws"' not in terraform_code:
        terraform_code = f'''
provider "aws" {{
  region = "us-east-1"
}}
{terraform_code}
'''

    # Update the file in the GitLab repository
    terraform_file.content = terraform_code
    terraform_file.save(branch='fix-terraform', commit_message='Fix: Add missing provider')

# Function to fix syntax errors in Terraform code
def update_terraform_syntax():
    # Fetch the Terraform file
    terraform_file = project.files.get(file_path='main.tf', ref='main')

    # Read the Terraform code
    terraform_code = terraform_file.decode()

    # Fix a simple syntax error (this is just an example, you'd need to handle real syntax errors)
    terraform_code = terraform_code.replace("provider aws", 'provider "aws"')

    # Update the file in the GitLab repository
    terraform_file.content = terraform_code
    terraform_file.save(branch='fix-terraform', commit_message='Fix: Correct Terraform syntax error')

# Function to create a pull request (merge request) in GitLab
def create_merge_request():
    # Create a new branch for the changes
    branch_name = "fix-terraform"
    project.branches.create({'branch': branch_name, 'ref': 'main'})

    # Create a Merge Request
    mr = project.mergerequests.create({
        'source_branch': branch_name,
        'target_branch': 'main',
        'title': 'Fix Terraform issues based on pipeline failure',
        'description': 'This merge request fixes Terraform issues identified in the pipeline logs.'
    })

    return f"Merge Request created successfully: {mr.web_url}"

# Initialize the LangChain agent for error analysis
error_analysis_tool = Tool(
    name="GitLab Pipeline Error Analyzer",
    func=analyze_pipeline_error,
    description="Fetches the latest pipeline status and analyzes the error logs if the pipeline fails."
)

# Initialize the LangChain agent for error fixing
error_fixing_tool = Tool(
    name="GitLab Terraform Error Fixer",
    func=fix_terraform_code,
    description="Attempts to fix Terraform issues based on pipeline error logs by modifying Terraform files and creating a pull request."
)

# Initialize the LLM (for example, OpenAI's GPT model)
llm = OpenAI(temperature=0.7)

# Initialize the LangChain agent with both tools
agent = initialize_agent([error_analysis_tool, error_fixing_tool], llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Example: Use the agent to analyze and fix the pipeline error
def analyze_and_fix_pipeline():
    response = agent.run({})
    return response

# Execute the analysis and apply the fixes
result = analyze_and_fix_pipeline()
print(result)

# Optionally, create the pull request if the fix was applied
mr_result = create_merge_request()
print(mr_result)
